In [34]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd
from surprise.model_selection import GridSearchCV

# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

In [14]:
df = pd.read_csv('전처리_토요일.csv')

C:\Users\ghg31\AppData\Local\Temp\ipykernel_25296\228154034.py:1: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('전처리_토요일.csv')


In [16]:
user_stats = df.groupby('user_id')['rating'].agg(['mean', 'std'])
biased_users = user_stats[(user_stats['std'] == 0) & (user_stats['mean'] == 5)].index
df_filtered = df[~df['user_id'].isin(biased_users)].copy()

In [18]:
 df_filtered.groupby("title").size().reset_index(name = "counts").sort_values(by = "counts", ascending = False)

,title,counts
372,나미야 잡화점의 기적 (양장),300
1513,오베라는 남자,283
69,7년의 밤,264
1429,엄마를 부탁해,261
665,라면을 끓이며,248
...,...,...
1626,"이 중에 1명, 여동생이 있다! 4",1
699,룬의 아이들 - 블러디드 2,1
1126,세계 미스터리 걸작선 01 : 살인자 외,1
1030,사쿠라장의 애완 그녀 3,1


In [20]:
 df_filtered[df_filtered["genre"] == "사회과학"]["title"].unique()

array(['어떻게 민주주의는 무너지는가', '사람, 장소, 환대', '국가는 왜 실패하는가 (리커버)', '예루살렘의 아이히만',
       '가난한 아이들은 어떻게 어른이 되는가', '공정하다는 착각', '왜 세계의 절반은 굶주리는가?', '지리의 힘',
       '페미니즘의 도전 (15주년 기념판)', '계속 가보겠습니다', '이재명의 나의 소년공 다이어리',
       '랭스로 되돌아가다', '군주론', '노예제와 사회적 죽음', '결국 국민이 합니다 : 이재명의 인생과 정치철학',
       '이재명의 길', '정치 성향은 어떻게 결정되는가', '공자가 죽어야 나라가 산다', '그들에게 국민은 없다',
       '전태일평전', '소유의 종말', '당신들의 대한민국 1', '폭력과 상스러움', '좌우는 있어도 위아래는 없다',
       '대화', '당신들의 대한민국 2', '신도 버린 사람들', '88만원 세대', '대한민국 개조론',
       '자본주의 역사 바로 알기', '후불제 민주주의', '유러피언 드림', '운명이다', '김대중 자서전 - 전2권',
       '불편해도 괜찮아', '이별에도 예의가 필요하다', '진보집권플랜', '닥치고 정치', '문재인의 운명 (반양장)',
       '달려라 정봉주', '분노하라', '국가란 무엇인가', '나는 꼼수다 뒷담화', '조국 현상을 말한다 - 개정판',
       '내가 걸은 만큼만 내 인생이다', '보수를 팝니다', '중생이 아프면 부처도 아프다',
       '그대 아직도 부자를 꿈꾸는가', '그들은 아는, 우리만 모르는', '우리가 잘못 산 게 아니었어',
       '21세기 자본 (양장)', '시골빵집에서 자본론을 굽다', '기록', '왜 가난한 사람들은 부자를 위해 투표하는가',
       '보수는 어떻게 지배하는가', '육식의 종말', '송곳 1~6 세트 - 전6권', '개인주의자 선언',
       '남자들은 자꾸 나를 가르치려 든다', '주기자의 

In [22]:
조국 = df_filtered[df_filtered["title"] == "조국의 시간"].index

In [47]:
df_filtered = df_filtered[~df_filtered.isin(조국)]

In [26]:
df_filtered = df_filtered[df_filtered["title"] != "조국의 시간"]

In [5]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_filtered[['user_id', 'isbn', 'rating']], reader)

In [30]:
df_filtered[df_filtered["title"] == "조국의 시간"]

,user_id,isbn,rating,review,title,writer,publisher,Pub_date,re_price,sa_price,genre,description,user_mean,rating_cen


In [32]:
df_filtered.to_csv("수요일_5점만 준사람제거_조국제거.csv")

In [6]:
trainset = data.build_full_trainset()

In [19]:
# GridSearchCV 수행 (데이터 전체 사용)
param_grid = {
    'n_epochs': [10, 20, 30],
    'lr_all': [0.001, 0.002, 0.003],
    'reg_all': [ 0.07, 0.09, 0.11],
    'n_factors': [10, 20, 50]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=10)
gs.fit(data)  


In [20]:
# 최고 RMSE 출력
print(gs.best_score['rmse'])
 
# 최고 RMSE의 parameter 출력
print(gs.best_params['rmse'])


0.8901553238459885
{'n_epochs': 30, 'lr_all': 0.003, 'reg_all': 0.07, 'n_factors': 10}


In [21]:
model = SVD( 
    n_epochs = 30,
    lr_all = 0.003,
    reg_all = 0.07,
    n_factors = 10)
model.fit(trainset)

predictions = model.test(trainset)
rmse = accuracy.rmse(predictions)

In [24]:
model = SVD( 
    n_epochs = 50,
    lr_all = 0.003,
    reg_all = 0.07,
    n_factors = 100)
model.fit(trainset)

In [28]:
param_grid = {
    'n_epochs': [30, 50, 80, 100, 120],
    'lr_all': [0.001, 0.002, 0.003, 0.004, 0.005],
    'reg_all': [ 0.03, 0.05, 0.07, 0.09, 0.11,],
    'n_factors': [10, 20, 50, 60, 70]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)
gs.fit(data)  


In [29]:
# 최고 RMSE 출력
print(gs.best_score['rmse'])
 
# 최고 RMSE의 parameter 출력
print(gs.best_params['rmse'])


0.8915344435741284
{'n_epochs': 120, 'lr_all': 0.001, 'reg_all': 0.09, 'n_factors': 10}


In [32]:
model = SVD( 
    n_epochs = 120,
    lr_all = 0.001,
    reg_all = 0.09,
    n_factors = 10)
model.fit(trainset)

In [33]:
def recommend_books(model, df_filtered, user_id, n=10, isbn_to_title=None):
    # 해당 사용자가 읽은 책 목록
    read_books = df_filtered[df_filtered['user_id'] == user_id]['isbn'].unique()
    all_books = df_filtered['isbn'].unique()
    unread_books = [isbn for isbn in all_books if isbn not in read_books]

    # unread_books에 대해 평점 예측
    preds = [model.predict(user_id, isbn) for isbn in unread_books]
    top_n = sorted(preds, key=lambda x: x.est, reverse=True)[:n]

    # 출력
    print(f"\n[사용자 {user_id} 추천 도서]")
    for pred in top_n:
        title = isbn_to_title.get(pred.iid, "제목 없음") if isbn_to_title else pred.iid
        print(f"책 제목: {title}, 예상 평점: {pred.est:.2f}")

In [34]:
# 책 제목 매핑
isbn_to_title = df_filtered[['isbn', 'title']].drop_duplicates().set_index('isbn')['title'].to_dict()

# 두 사용자에게 추천
recommend_books(model, df_filtered, '보슬비', isbn_to_title=isbn_to_title)
recommend_books(model, df_filtered, '책읽는나무', isbn_to_title=isbn_to_title)



[사용자 보슬비 추천 도서]
책 제목: 조국의 시간, 예상 평점: 4.78
책 제목: 코스모스, 예상 평점: 4.75
책 제목: 명상록 (그리스어 원전 완역본), 예상 평점: 4.74
책 제목: 산하엽, 예상 평점: 4.71
책 제목: 헤어질 결심 각본, 예상 평점: 4.71
책 제목: 아픔이 길이 되려면, 예상 평점: 4.70
책 제목: 수박 수영장, 예상 평점: 4.69
책 제목: 대화, 예상 평점: 4.68
책 제목: 나는 소망한다 내게 금지된 것을, 예상 평점: 4.68
책 제목: 송곳 1~6 세트 - 전6권, 예상 평점: 4.68

[사용자 책읽는나무 추천 도서]
책 제목: 헤어질 결심 각본, 예상 평점: 4.99
책 제목: 딕테, 예상 평점: 4.99
책 제목: 코스모스, 예상 평점: 4.98
책 제목: 산하엽, 예상 평점: 4.97
책 제목: 명상록 (그리스어 원전 완역본), 예상 평점: 4.96
책 제목: 조국의 시간, 예상 평점: 4.95
책 제목: 나는 소망한다 내게 금지된 것을, 예상 평점: 4.95
책 제목: 대화, 예상 평점: 4.95
책 제목: 죄와 벌 1, 예상 평점: 4.94
책 제목: 금요일엔 돌아오렴, 예상 평점: 4.94


In [10]:
# 두 유저가 실제로 어떤 책을 읽었는지 비교
user1_books = set(df_filtered[df_filtered['user_id'] == '보슬비']['isbn'])
user2_books = set(df_filtered[df_filtered['user_id'] == '책읽는나무']['isbn'])

print("공통으로 읽은 책 수:", len(user1_books & user2_books))
print("각자 읽은 책 수:", len(user1_books), len(user2_books))


공통으로 읽은 책 수: 5
각자 읽은 책 수: 40 109


In [ ]:
# double_rating

In [67]:
df_filtered["double_rating"] = df_filtered["rating"]*2

In [70]:
reader = Reader(rating_scale=(1, 10))
data2 = Dataset.load_from_df(df_filtered[['user_id', 'isbn', 'double_rating']], reader)

In [71]:
trainset2 = data2.build_full_trainset()

In [72]:
param_grid = {
    'n_epochs': [30, 50, 80, 100, 120],
    'lr_all': [0.001, 0.002, 0.003, 0.004, 0.005],
    'reg_all': [ 0.03, 0.05, 0.07, 0.09, 0.11,],
    'n_factors': [10, 20, 50, 60, 70]}

gs2 = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)
gs2.fit(data2)  


In [74]:
# 최고 RMSE 출력
print(gs2.best_score['rmse'])
 
# 최고 RMSE의 parameter 출력
print(gs2.best_params['rmse'])


1.7854908814521901
{'n_epochs': 30, 'lr_all': 0.004, 'reg_all': 0.11, 'n_factors': 10}


In [75]:
model2 = SVD( 
    n_epochs = 30,
    lr_all = 0.004,
    reg_all = 0.11,
    n_factors = 10)
model2.fit(trainset2)

In [76]:
def recommend_books(model, df_filtered, user_id, n=10, isbn_to_title=None):
    # 해당 사용자가 읽은 책 목록
    read_books = df_filtered[df_filtered['user_id'] == user_id]['isbn'].unique()
    all_books = df_filtered['isbn'].unique()
    unread_books = [isbn for isbn in all_books if isbn not in read_books]

    # unread_books에 대해 평점 예측
    preds = [model2.predict(user_id, isbn) for isbn in unread_books]
    top_n = sorted(preds, key=lambda x: x.est, reverse=True)[:n]

    # 출력
    print(f"\n[사용자 {user_id} 추천 도서]")
    for pred in top_n:
        title = isbn_to_title.get(pred.iid, "제목 없음") if isbn_to_title else pred.iid
        print(f"책 제목: {title}, 예상 평점: {pred.est:.2f}")

In [78]:
# 책 제목 매핑
isbn_to_title = df_filtered[['isbn', 'title']].drop_duplicates().set_index('isbn')['title'].to_dict()

# 두 사용자에게 추천
recommend_books(model, df_filtered, '보슬비', isbn_to_title=isbn_to_title)
recommend_books(model, df_filtered, '책읽는나무', isbn_to_title=isbn_to_title)



[사용자 보슬비 추천 도서]
책 제목: 코스모스, 예상 평점: 9.46
책 제목: 헤어질 결심 각본, 예상 평점: 9.43
책 제목: 도스토옙스키 컬렉션 (도스토옙스키 탄생 200주년 기념판) - 전11권, 예상 평점: 9.43
책 제목: 딕테, 예상 평점: 9.43
책 제목: 명상록 (그리스어 원전 완역본), 예상 평점: 9.42
책 제목: 조국의 시간, 예상 평점: 9.40
책 제목: 산하엽, 예상 평점: 9.40
책 제목: 죄와 벌 1, 예상 평점: 9.37
책 제목: 대화, 예상 평점: 9.37
책 제목: 나의 아저씨 1~2 세트 - 전2권, 예상 평점: 9.36

[사용자 책읽는나무 추천 도서]
책 제목: 명상록 (그리스어 원전 완역본), 예상 평점: 10.00
책 제목: 헤어질 결심 각본, 예상 평점: 10.00
책 제목: 코스모스, 예상 평점: 9.98
책 제목: 조국의 시간, 예상 평점: 9.95
책 제목: 산하엽, 예상 평점: 9.92
책 제목: 금요일엔 돌아오렴, 예상 평점: 9.90
책 제목: 어린 왕자, 예상 평점: 9.89
책 제목: 딕테, 예상 평점: 9.89
책 제목: 아픔이 길이 되려면, 예상 평점: 9.87
책 제목: 나의 아저씨 1~2 세트 - 전2권, 예상 평점: 9.87


In [31]:
df_filtered[df_filtered["user_id"] == '후애(厚愛)']

,user_id,isbn,rating,review,title,writer,publisher,Pub_date,re_price,sa_price,genre,description,user_mean,rating_cen


In [32]:
df_filtered

,user_id,isbn,rating,review,title,writer,publisher,Pub_date,re_price,sa_price,genre,description,user_mean,rating_cen
0,보슬비,9788936434120,5,"어떻게 얻은 민주주의인데... 우리의 힘은 ‘잊지말기‘, ‘기억하기‘ 그래서 ‘지켜내기‘",소년이 온다,한강 지음,창비,20140519.0,"15,000","13,500",소설/시/희곡,"말라파르테 문학상, 만해문학상 수상작 \n우리 시대의 소설 『소년이 온다』\n\n2...",4.300000,0.700000
1,책읽는나무,9788936434120,5,"아픔과 슬픔은 경험해봐야 크게 공감할 수 있다지만, 글을 읽으면서도 이미 눈은 슬...",소년이 온다,한강 지음,창비,20140519.0,"15,000","13,500",소설/시/희곡,"말라파르테 문학상, 만해문학상 수상작 \n우리 시대의 소설 『소년이 온다』\n\n2...",4.412844,0.587156
2,코코몽,9788936434120,5,역사를 잊은 민족에게 미래는 없다.,소년이 온다,한강 지음,창비,20140519.0,"15,000","13,500",소설/시/희곡,"말라파르테 문학상, 만해문학상 수상작 \n우리 시대의 소설 『소년이 온다』\n\n2...",4.666667,0.333333
3,the,9788936434120,5,출근길에 지하철에서 숨죽여 오열하며 마지막 장을 다 읽었습니다. 고통 속에서 한 자...,소년이 온다,한강 지음,창비,20140519.0,"15,000","13,500",소설/시/희곡,"말라파르테 문학상, 만해문학상 수상작 \n우리 시대의 소설 『소년이 온다』\n\n2...",4.142857,0.857143
4,appletreeje,9788936434120,5,습자지에 핏망울이 꽃처럼 스며드는 이야기. 새의 영혼같이 나직나직 들려주는 이 책을...,소년이 온다,한강 지음,창비,20140519.0,"15,000","13,500",소설/시/희곡,"말라파르테 문학상, 만해문학상 수상작 \n우리 시대의 소설 『소년이 온다』\n\n2...",4.941176,0.058824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66003,sophie833,9788934935759,5,별로 기대를 안했는데 읽다보니 꽤 재미있네요. 우리나라가 이제 약소국은 아니지만아직...,변방에서 중심으로,"문재인 지음, 최종건 대담",김영사,20240518.0,33000,29700,사회과학,"퇴임 2주년 첫 회고록!\n위기에서 희망으로, 대립에서 번영으로, 변방에서 중심으로...",4.666667,0.333333
66004,아라,9788934935759,5,읽는 것만으로도 가슴이 벅차오른다. 문재인 정부가 지향했던 가치와 혼신의 힘을 다해...,변방에서 중심으로,"문재인 지음, 최종건 대담",김영사,20240518.0,33000,29700,사회과학,"퇴임 2주년 첫 회고록!\n위기에서 희망으로, 대립에서 번영으로, 변방에서 중심으로...",3.575342,1.424658
66007,azrael,9788934935759,5,책을 읽으면서 새록새록 기억이 되살아 났습니다...이 땅에 다시 전쟁이 일어나서는 ...,변방에서 중심으로,"문재인 지음, 최종건 대담",김영사,20240518.0,33000,29700,사회과학,"퇴임 2주년 첫 회고록!\n위기에서 희망으로, 대립에서 번영으로, 변방에서 중심으로...",4.857143,0.142857
66008,김정희,9788934935759,5,당신이 그립습니다..ㅠㅠ 여러분야에서 애써주신 대통령님 고생 많으셨습니다!!!,변방에서 중심으로,"문재인 지음, 최종건 대담",김영사,20240518.0,33000,29700,사회과학,"퇴임 2주년 첫 회고록!\n위기에서 희망으로, 대립에서 번영으로, 변방에서 중심으로...",4.833333,0.166667


In [11]:


# # 3. 책 제목 딕셔너리
# isbn_to_title = df_filtered[['isbn', 'title']].drop_duplicates().set_index('isbn')['title'].to_dict()

# # 4. phi-2 모델 로드
# llm_model_name = "microsoft/phi-2"
# tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
# llm_model = AutoModelForCausalLM.from_pretrained(llm_model_name, torch_dtype=torch.float32).to("cpu")

# # 5. 추천 이유 생성 함수
# def generate_explanation(user_id, book_title, est_rating):
#     prompt = f"""사용자 "{user_id}"님께 "{book_title}" 도서를 추천하는 이유는 다음과 같습니다.
# 예상 평점은 {est_rating:.2f}점입니다.
# 추천 이유:"""

#     inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
#     outputs = llm_model.generate(
#         **inputs,
#         max_new_tokens=80,
#         pad_token_id=tokenizer.eos_token_id,
#         do_sample=True,
#         temperature=0.7,
#         top_k=50,
#         top_p=0.95
#     )
#     result = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     explanation = result.split("추천 이유:")[-1].strip()
#     return explanation

# # 6. 추천 함수
# def recommend_books_with_explanation(model, df_filtered, user_id, n=1, isbn_to_title=None):
#     read_books = df_filtered[df_filtered['user_id'] == user_id]['isbn'].unique()
#     all_books = df_filtered['isbn'].unique()
#     unread_books = [isbn for isbn in all_books if isbn not in read_books]

#     preds = [model.predict(user_id, isbn) for isbn in unread_books]
#     top_n = sorted(preds, key=lambda x: x.est, reverse=True)[:n]

#     print(f"\n[{user_id}님 추천 도서]")
#     for pred in top_n:
#         title = isbn_to_title.get(pred.iid, "제목 없음") if isbn_to_title else pred.iid
#         explanation = generate_explanation(user_id, title, pred.est)
#         print(f"책 제목: {title}, 예상 평점: {pred.est:.2f}")
#         print(f"추천 이유: {explanation}")

# # 7. 테스트 실행
# recommend_books_with_explanation(model, df_filtered, "보슬비", n=1, isbn_to_title=isbn_to_title)


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

ImportError: 
AutoModelForCausalLM requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [12]:
# # 책 제목/설명 딕셔너리
# isbn_to_title = df_filtered[['isbn', 'title']].drop_duplicates().set_index('isbn')['title'].to_dict()
# isbn_to_desc = df_filtered[['isbn', 'description']].drop_duplicates().set_index('isbn')['description'].to_dict()

# # LLM (phi-2) 불러오기
# tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
# llm_model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype=torch.float32).to("cpu")

# # 설명 생성 함수
# def generate_explanation_with_context(user_id, book_title, book_desc, est_rating):
#     prompt = f"""책 제목: {book_title}
# 책 설명: {book_desc}

# 사용자 "{user_id}"님께 이 책을 추천하는 이유는 다음과 같습니다.
# 예상 평점은 {est_rating:.2f}점입니다.
# 추천 이유:"""
    
#     inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
#     outputs = llm_model.generate(
#         **inputs,
#         max_new_tokens=100,
#         pad_token_id=tokenizer.eos_token_id,
#         do_sample=True,
#         temperature=0.7,
#         top_k=50,
#         top_p=0.95
#     )
#     result = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     explanation = result.split("추천 이유:")[-1].strip()
#     return explanation

# # 추천 함수
# def recommend_books_with_reason(model, df_filtered, user_id, n=1, isbn_to_title=None, isbn_to_desc=None):
#     read_books = df_filtered[df_filtered['user_id'] == user_id]['isbn'].unique()
#     all_books = df_filtered['isbn'].unique()
#     unread_books = [isbn for isbn in all_books if isbn not in read_books]

#     preds = [model.predict(user_id, isbn) for isbn in unread_books]
#     top_n = sorted(preds, key=lambda x: x.est, reverse=True)[:n]

#     print(f"\n[{user_id}님 추천 도서]")
#     for pred in top_n:
#         title = isbn_to_title.get(pred.iid, "제목 없음")
#         desc = isbn_to_desc.get(pred.iid, "설명이 없습니다.")
#         explanation = generate_explanation_with_context(user_id, title, desc, pred.est)

#         print(f" 책 제목: {title}")
#         print(f" 예상 평점: {pred.est:.2f}")
#         print(f" 추천 이유: {explanation}")


# recommend_books_with_reason(model, df_filtered, "책읽는나무", n=1, isbn_to_title=isbn_to_title, isbn_to_desc=isbn_to_desc)

ImportError: 
AutoModelForCausalLM requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
